In [ ]:
dir_data = '../data'

token = 'YOUR AUTHORIZED TOKEN'

In [ ]:
import sys
sys.path.append('../')

from suncapturer.utils import Requestor

import os
import pandas as pd

from datetime import datetime


In [ ]:
requestor = Requestor(token)

In [ ]:
path_gens = os.path.join(dir_data, 'gens_collected.csv')
if not os.path.exists(path_gens):
    gens_collected = pd.DataFrame()
    start = '2022-10-24'
else:
    gens_collected = pd.read_csv(path_gens)
    start = gens_collected['time'].max().split(' ')[0]

In [ ]:
print(len(gens_collected))

In [ ]:
start = datetime.strptime(start, '%Y-%m-%d').date()
today = datetime.now().date()

dates = pd.date_range(start, today, freq='1d')[:-1]
for date in dates:
    gens = requestor.get_pv_gens(str(date.date()))
    gens.columns = ['id', 'time', 'amount']
    
    gens['time'] = pd.to_datetime(gens['time'])
    gens['time'] = gens['time'].dt.tz_convert(tz='Asia/Seoul')
    
    gens = gens.sort_values(['id', 'time']).reset_index(drop=True)
    
    gens_collected = pd.concat([gens_collected, gens], axis=0, ignore_index=True)

# gens_collected.to_csv(path_gens, index=False)

In [ ]:
print(len(gens_collected))

In [ ]:
path_fcst = os.path.join(dir_data, 'fcst_collected.csv')
if not os.path.exists(path_fcst):
    start = '2022-10-24'
    fcst_collected = pd.DataFrame()
else:
    fcst_collected = pd.read_csv(path_fcst)
    start = fcst_collected['fcst_time'].max()
    start = datetime.fromisoformat(start).date()

In [ ]:
len(fcst_collected)

In [ ]:
today = datetime.now().date()

dates = pd.date_range(start, today, freq='1d')[:-1]
for date in dates:
    for idx in range(1, 22):
        fcst = requestor.get_forecasts(1, str(date.date()), 16)

        fcst['fcst_time'] = pd.to_datetime(fcst['fcst_time'])
        fcst['fcst_time'] = fcst['fcst_time'].dt.tz_convert(tz='Asia/Seoul')
        
        fcst['time'] = pd.to_datetime(fcst['time'])
        fcst['time'] = fcst['time'].dt.tz_convert(tz='Asia/Seoul')
        
        columns = list(fcst.columns)
        columns = ['id'] + columns
        
        fcst['id'] = idx
        fcst = fcst[columns]
        
        fcst_collected = pd.concat([fcst_collected, fcst], axis=0, ignore_index=True)

fcst_collected.to_csv(path_fcst, index=False)

In [ ]:
len(fcst_collected)